In [1]:
import os
import math

In [2]:
def get_label(longname):
    """
    Convert a long name to a short version
    """
    filename = os.path.basename(longname)  # Extract filename only
    return filename if len(filename) <= 10 else filename[:7] + '...'

def get_rel_paths(here_path, root_path):
    """
    Given the current full path and the root path, return the
    relative path to the directory above, and the relative path
    to this full path
    """
    rel_path = os.path.relpath(here_path, root_path) # Compute relative path

    if here_path == root_path:  # Special case: same directory
        return ('..', '.')
    
    parts = rel_path.split(os.sep)  # Split the path into components

    if len(parts) == 1:  # Special case: Direct child of parent_dir
        return ('.', parts[0])
    
    return (parts[0], rel_path)  # General case


def remove_excluded_subdirs(dir_list):
    """
    For os.path.walk(), we can restrict the branches that get traversed by changing
    the list which walk() returns as 'subdirs'.  Remember that when we are editing
    that list, we are actually changing the memory representation inside the walk()
    generator, so the semantics are a little tricky.  For example, we can't simultaneously
    loop over the list and edit it.  I'll give you a working function for this task
    to avoid confusion.
    
    This function removes 'reveal.js', '.git', and anything starting with '_' from the list.
    """
    more_names_to_remove = [elt for elt in dir_list if elt.startswith('_')][:]
    for nm in ['reveal.js', '.git'] + more_names_to_remove:
        if nm in dir_list:
            dir_list.remove(nm)

def show_this_leaf(nm):
    """
    Use this function to return False for leaf node names you don't want to draw.
    """
    # Exclude nodes starting with '.' or '_' or ending with '~' or starting with '#'
    if nm.startswith(('.', '_')) or nm.endswith('~') or nm.startswith('#'):
        return False  # Don't show this leaf node

    return True  # If the node passes the filter, show it

seen_edges = set()

class Node():
    def __init__(self, parent_node, name):
        """
        parent_node should be the Node instance of the parent.
        
        name is a long name, like a full relative path.  It needs to be
        unique for all nodes in the tree.
        
        Note how we make a connection to the parent node when this node is created.
        """
        self.parent = parent_node
        if self.parent is not None:
            self.parent._add_kid(self)
        self.name = name
        self.label = get_label(self.name)
        self.kids = []
        self.descendant_count = 0
    def _add_kid(self, kid_node):
        self.kids.append(kid_node)
    def add_descendant_to_all_ancestors(self):
        """
        Modify this function so that when it is called for a node's parent,
        that parent and all ancestors get their descendant_count incremented.
        """

        node = self  # Start at self 
        while node is not None:
            node.descendant_count += 1  # Increment count for ancestors
            node = node.parent  # Move up to the next ancestor
    
    def count_descendants(self):
        """Recursive method to count all descendants."""
        self.descendant_count = len(self.kids)  # Reset descendant count
        # print(f"Counting descendants for node {self.name}, initial count: {self.descendant_count}")
        for kid in self.kids:
            self.descendant_count += kid.count_descendants()  # Recursively count
            # print(f"Node {self.name} has {self.descendant_count} descendants.")
        return self.descendant_count
    
    def get_node_attributes(self):
        """
        Returns attributes for the node based on the type of the node and its filename.
        """
        # Determine shape based on whether it's a leaf node or not
        shape = "rect" if len(self.kids) == 0 else "ellipse"  # Leaf nodes are rect, others are ellipse
        
        # Determine color based on the file extension
        if self.name.endswith(('.png', '.jpg', '.svg')):
            fillcolor = "blue"
        else:
            fillcolor = "lightgray"
        
        # Return the attributes in a format suitable for Graphviz
        return {
            "shape": shape,
            "fillcolor": fillcolor,
            "style": "filled"  # This ensures the background is filled with the color
        }

    
    def write_node(self, indent=0):
        """Write the DOT code to define a single node."""
        # Determine the shape and fillcolor based on the node type
        attributes = self.get_node_attributes()  # Get the node attributes
        # Update the node's DOT attributes with shape, fillcolor, and style
        print(f'{indent*" "}"{self.name}" [label="{self.label}" shape={attributes["shape"]} ' 
            f'fillcolor={attributes["fillcolor"]} style={attributes["style"]}];')

    
    def traverse_node_defs(self, indent=0):
        """
        Write the DOT code that defines this Node and all its descendants.
        
        Here and in traverse_edge_defs(), 'indent' just helps with formatting
        when writing out the DOT code.
        """
        self.write_node(indent=indent) # Write the current node's definition
        for kid in self.kids:
            kid.traverse_node_defs(indent+4) # Recursively write definitions for the children

    def write_incoming_edge(self, this_parent, indent=0):
            """
            Calculate edge width based on the square root of descendants.
            """
            if this_parent.descendant_count > 0:
                width = round(this_parent.descendant_count ** 0.5, 2)
            else:
                width = 1  # Default width if no descendants
            edge = (this_parent.name, self.name)
            if edge not in seen_edges:
                # Compute the penwidth based on the square root of the descendant count.
                penwidth = math.sqrt(self.descendant_count) if self.descendant_count > 0 else 1
                print(f'{indent*" "}"{this_parent.name}" -> "{self.name}" [penwidth={penwidth}];')
                seen_edges.add(edge)
            

    def traverse_edge_defs(self, indent=0):
        """
        Write the DOT code that defines the incoming edges for this Node and
        all its descendants.
        
        Here and in traverse_node_defs(), 'indent' just helps with formatting
        when writing out the DOT code.
        """
        for kid in self.kids:
            kid.write_incoming_edge(self, indent=indent + 4)  # Write edges
            kid.traverse_edge_defs(indent + 4)  # Recursively process children


# Maintain a dictionary of Nodes so that we can find them by name.  Use
# paths relative to the root as keys- so the very first key is just '.'
nodes = {}
root_path = 'CMU-MS-DAS-Vis-S25/docs'
root_node = Node(None, '.')
nodes['.'] = root_node

# Walk the tree
for dirname, subdirs, files in os.walk(root_path):
    remove_excluded_subdirs(subdirs)
    rel_dir_path, rel_path = get_rel_paths(dirname, root_path)
    if rel_path in nodes:
        # This happens on the very first node
        dir_node = nodes[rel_path]
    else:
        assert rel_dir_path in nodes
        dir_node= Node(nodes[rel_dir_path], rel_path)
        nodes[rel_path] = dir_node

    # Add nodes for all the children of this dir
    for file in files:
        if show_this_leaf(file):
            full_path = os.path.join(dirname, file)
            ignore_this, rel_path = get_rel_paths(full_path, root_path)
            this_node = Node(dir_node, rel_path)
            nodes[rel_path] = this_node

# Calculate number of descendants for all nodes
for node in nodes:
    nodes[node].add_descendant_to_all_ancestors()

# Write out the Dot code
print("digraph {")
root_node.traverse_node_defs()
root_node.traverse_edge_defs()
print("}")

digraph {
"." [label="." shape=ellipse fillcolor=lightgray style=filled];
    "assignment_matplotlib.html" [label="assignm..." shape=rect fillcolor=lightgray style=filled];
    "assignment_maps_body.md" [label="assignm..." shape=rect fillcolor=lightgray style=filled];
    "idioms_for_statistics.html" [label="idioms_..." shape=rect fillcolor=lightgray style=filled];
    "assignment_ipywidgets_body.md" [label="assignm..." shape=rect fillcolor=lightgray style=filled];
    "assignment_ggplot.html" [label="assignm..." shape=rect fillcolor=lightgray style=filled];
    "graphs_with_nodes_and_edges.html" [label="graphs_..." shape=rect fillcolor=lightgray style=filled];
    "overview.md" [label="overvie..." shape=rect fillcolor=lightgray style=filled];
    "git_and_github.html" [label="git_and..." shape=rect fillcolor=lightgray style=filled];
    "ipywidgets_intro.html" [label="ipywidg..." shape=rect fillcolor=lightgray style=filled];
    "dashboards_body.md" [label="dashboa..." shape=rect fill